# AKSSF Hydrography
## Construct Synthetic Stream Network and Catchments using TauDEM tools and Timm's 10m composite DEM clipped to Study Area

### Define Hydroprocessing Function
Create synthetic stream network and calculate stream reach and watersheds(catchments) using TauDEM tools
**May want to burn in NHD streams to force flow direction, can consider after examining results**

1. Clip input 10 meter composite DEM to study area and fill pits
2. Calculate D8 flow direction grid
    * This process takes the longest
3. Calculate D8 contributing area
4. Generate stream grid using Peuker Douglass Stream Definition
    * Accumulation threshold = 75
    * Minimum threshold = 20
    * Maximum threshold = 500
    * Number threshold values = 15
5. Process stream reach and watersheds
6. Export watershed grid to shapefile

### Inputs:
1. study_area: A polygon feature class with the field "Processing_Region" that has been coded to designate unique
watersheds to be processed iteratively.

2. TauLoc: path to TauDEM toolbox on local machine <a href = "https://hydrology.usu.edu/taudem/taudem5/"> TauDEM
</a> must be installed.

3. DEM:  path to 10 meter composite DEM covering study area.  DEM will be clipped to the study area polygon during
processing.

4. datafolder: path to Output folder where data will be saved.


In [1]:
def hydroprocess(**kwargs):
    # Import packages
    import os
    import arcpy
    import datetime
    import time

    """
    Description: Creates synthetic stream network and catchments from input DEM
    :param Input_Number_of_Processes: 10
    :param Input_DEM: demclip #Dem clipped to study area
    :param Check_for_edge_contamination:False #Dem clipped to watershed boundary so keep false
    :param Outputs: region_out # name of processing region
    :param _Name_catchments_shp: catchname
    :param Output_Stream_Reach_file: streamname
    :return:
    """

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Import TauDEM tools
    arcpy.ImportToolbox(TauLoc)

    # Use two thirds of cores on processes that can be split.
    arcpy.env.parallelProcessingFactor = "66%"

    # Parse key word argument inputs
    outfol = kwargs['outfol']
    Input_DEM = kwargs['rasclip']
    region = kwargs['region']
    Input_Number_of_Processes = kwargs['Input_Number_of_Processes']
    Check_for_edge_contamination = kwargs['Check_for_edge_contamination']


    # Process: Pit Remove
    DEM_fillname = region + "_fel.tif"
    DEM_fill = os.path.join(outfol, DEM_fillname)

    if not os.path.exists(DEM_fill):
        # Start timing function
        iteration_start = time.time()
        arcpy.PitRemove(Input_Elevation_Grid=Input_DEM, Fill_Considering_only_4_way_neighbors=False,
                         Input_Depression_Mask_Grid="", Input_Number_of_Processes=Input_Number_of_Processes,
                         Output_Pit_Removed_Elevation_Grid=DEM_fill)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Pits filled at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Pit Filled Elevation Grid already exists ")
        print('----------')
    DEM_fill = arcpy.Raster(DEM_fill)

    # Process: D8 Flow Directions (D8 Flow Directions) ()
    _Name_p_tifname = region + "_p.tif"
    _Name_sd8_tifname = region + "_sd8.tif"
    _Name_p_tif = os.path.join(outfol, _Name_p_tifname)
    _Name_sd8_tif = os.path.join(outfol,_Name_sd8_tifname)
    if not os.path.exists(_Name_p_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.D8FlowDir(Input_Pit_Filled_Elevation_Grid=DEM_fill,
                         Input_Number_of_Processes=Input_Number_of_Processes,
                         Output_D8_Flow_Direction_Grid=_Name_p_tif, Output_D8_Slope_Grid=_Name_sd8_tif)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Flow Direction calculated at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Flow Direction Grid already exists")
        print('----------')
    _Name_p_tif = arcpy.Raster(_Name_p_tif)
    _Name_sd8_tif = arcpy.Raster(_Name_sd8_tif)

    # Process: D8 Contributing Area (D8 Contributing Area) ()
    _Name_ad8_tifname = region + "_ad8.tif"
    _Name_ad8_tif = os.path.join(outfol, _Name_ad8_tifname)
    if not os.path.exists(_Name_ad8_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.D8ContributingArea(Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                  Input_Outlets="", Input_Weight_Grid="",
                                  Check_for_edge_contamination=Check_for_edge_contamination,
                                  Input_Number_of_Processes=Input_Number_of_Processes,
                                  Output_D8_Contributing_Area_Grid=_Name_ad8_tif)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'D8 Contributing area calculated at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print ('D8 Contributing Area Grid Already Exists')
        print('----------')
    _Name_ad8_tif = arcpy.Raster(_Name_ad8_tif)

    # Process: Peuker Douglas Stream Definition (Peuker Douglas Stream Definition) ()
    _Name_ss_tifname = region + "_ss.tif"
    _Name_ssa_tifname = region + "_ssa.tif"
    _Name_src_tifname = region + "_src.tif"
    _Name_drp_txtname = region + "_drp.txt"
    _Name_ss_tif = os.path.join(outfol, _Name_ss_tifname)
    _Name_ssa_tif = os.path.join(outfol, _Name_ssa_tifname)
    _Name_src_tif = os.path.join(outfol, _Name_src_tifname)
    _Name_drp_txt = os.path.join(outfol, _Name_drp_txtname)

    if not os.path.exists(_Name_src_tif):
        # Start timing function
        iteration_start = time.time()
        arcpy.PeukerDouglasStreamDef(Input_Elevation_Grid=DEM_fill,
                                      Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                      Weight_Center=0.4, Weight_Side=0.1, Weight_Diagonal=0.05,
                                      Accumulation_Threshold=75,
                                      Check_for_Edge_Contamination=Check_for_edge_contamination,
                                      Input_Outlets="", Input_Mask_Grid="",
                                      Input_D8_Contributing_Area_for_Drop_Analysis=_Name_ad8_tif,
                                      Input_Number_of_Processes=Input_Number_of_Processes,
                                      Output_Stream_Source_Grid=_Name_ss_tif,
                                      Output_Accumulated_Stream_Source_Grid=_Name_ssa_tif,
                                      Output_Stream_Raster_Grid=_Name_src_tif,
                                      Output_Drop_Analysis_Table=_Name_drp_txt,
                                      Use_the_range_below_to_automatically_select_threshold_by_drop_analysis=True,
                                      Minimum_Threshold_Value=20, Maximum_Threshold_Value=500,
                                      Number_of_Threshold_Values=15,
                                      Use_Logarithmic_spacing_for_threshold_values=True)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Stream Definition completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print('Streams Already Defined')
        print('----------')
    _Name_ss_tif = arcpy.Raster(_Name_ss_tif)
    _Name_ssa_tif = arcpy.Raster(_Name_ssa_tif)
    _Name_src_tif = arcpy.Raster(_Name_src_tif)

    # Process: Stream Reach And Watershed (Stream Reach And Watershed) ()
    Output_Stream_Reach_file = region + '_stream_reach.shp'
    Output_Stream_Order_Gridname = region + "_ord.tif"
    Output_Network_Connectivity_Treename = region + "_tree.txt"
    Output_Network_Coordinatesname = region + "_coord.txt"
    Output_Watershed_Gridname = region + "_w.tif"
    Output_Stream_Order_Grid = os.path.join(outfol, Output_Stream_Order_Gridname)
    Output_Network_Connectivity_Tree =  os.path.join(outfol, Output_Network_Connectivity_Treename)
    Output_Network_Coordinates = os.path.join(outfol, Output_Network_Coordinatesname)
    Output_Watershed_Grid = os.path.join(outfol, Output_Watershed_Gridname)

    if not os.path.exists(Output_Stream_Order_Grid):
        # Start timing function
        iteration_start = time.time()
        arcpy.StreamReachAndWatershed(Input_Pit_Filled_Elevation_Grid=DEM_fill,
                                       Input_D8_Flow_Direction_Grid=_Name_p_tif,
                                       Input_D8_Drainage_Area=_Name_ad8_tif,
                                       Input_Stream_Raster_Grid=_Name_src_tif,
                                       Input_Outlets="", Delineate_Single_Watershed=False,
                                       Input_Number_of_Processes=Input_Number_of_Processes,
                                       Output_Stream_Order_Grid=Output_Stream_Order_Grid,
                                       Output_Network_Connectivity_Tree=Output_Network_Connectivity_Tree,
                                       Output_Network_Coordinates=Output_Network_Coordinates,
                                       Output_Stream_Reach_file=Output_Stream_Reach_file,
                                       Output_Watershed_Grid=Output_Watershed_Grid)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Stream Reach and Watershed completed at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print ('Stream Reach and Watershed already identified')
        print('----------')
    Output_Stream_Order_Grid = arcpy.Raster(Output_Stream_Order_Grid)
    Output_Watershed_Grid = arcpy.Raster(Output_Watershed_Grid)

    # Process: Watershed Grid To Shapefile
    _Name_catchments_shpname = region + "_watersheds.shp"
    _Name_catchments_shp = os.path.join(outfol, _Name_catchments_shpname)
    if not os.path.exists(_Name_catchments_shp):
        # Start timing function
        iteration_start = time.time()
        arcpy.WaterShedGridToShapefile(Input_Watershed_Grid=Output_Watershed_Grid,
                                       Output_Watershed_Shapefile=_Name_catchments_shp)
        # End timing
        iteration_end = time.time()
        iteration_elapsed = int(iteration_end - iteration_start)
        iteration_success_time = datetime.datetime.now()
        # Report success
        print(f'Watershed Exported at {iteration_success_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=iteration_elapsed)})')
        print('----------')
    else:
        print("Watershed Grid already exported to shapefile")
        print('----------')

# Set Inputs


In [4]:
import arcpy
import os

# path to TauDEM toolbox on local machine
TauLoc = r"C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx"

# 10m Composite DEM Clipped to Study Area
DEM = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\topography\\AKSSF_Elevation_Composite_10m.tif"

# Input feature class describing study area and processing regions that can be iterated over
study_area = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_Hydrography.gdb\\AKSSF_Bristol_Bay"

# Folder to store outputs
datafolder = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs\\TauDEM"

inputs = [TauLoc, DEM, study_area, datafolder]
for input in inputs:
    if not os.path.exists(input):
        print('Please check that the path for {} is correct'.format(input))
        print('\t---------')
    else:
        print('Input {} set'.format(input))
        print('\t---------')





Input C:\Program Files\TauDEM\TauDEM5Arc\TauDEM Tools.tbx set
	---------
Input C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\topography\\AKSSF_Elevation_Composite_10m.tif set
	---------
Please check that the path for C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\AKSSF_Hydrography.gdb\\AKSSF_Bristol_Bay is correct
	---------
Input C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs\\TauDEM set
	---------


# Begin Processing

In [3]:
import datetime
import time
arcpy.env.overwriteOutput = True
arcpy.env.snapRaster = DEM

proc_list = ['Egegik', 'Naknek']

# Start timing function
processStart = time.time()

for region in proc_list:
    outfol = os.path.join(datafolder, region)
    if not os.path.exists(outfol):
        os.makedirs(outfol)
        print('{} output location created at {}'.format(region, outfol))
        print('\t----------')
    else:
        print('{} output location already exists at {}'.format(region, outfol))
        print('\t----------')

    arcpy.env.workspace = outfol
    # Create roi layer
    layer = region + "_layer"
    print(layer)
    print('\t----------')

    # Create expression
    reg = "'" + region + "'"
    print(reg)
    print('\t----------')

    exp = "Processing_Region = {}".format(reg)
    print(exp)
    print('\t----------')
    rasclip = os.path.join(outfol, region + "_10mCompDEMClip.tif")
    roilayer = arcpy.MakeFeatureLayer_management(study_area, layer, exp)
    if not os.path.exists(rasclip):
        print("Clipping DEM to {} region ".format(region))
        print("\t-----------")
        demclip = arcpy.management.Clip(in_raster = DEM,rectangle = "", out_raster = rasclip,
                                        in_template_dataset = roilayer, nodata_value = -32768,
                                        clipping_geometry = "ClippingGeometry",
                                        maintain_clipping_extent = "NO_MAINTAIN_EXTENT")
    else:
        print("{} Processing Region already exists")
        print("\t-----------")
    print("Begin Hydro Processing at")
    # Keyword arguments for hydro processing
    hydroprocess_kwargs = {'outfol': outfol,
                       'rasclip':rasclip,
                       'region':region,
                       'Input_Number_of_Processes':10,
                       'Check_for_edge_contamination':False
                      }

    hydroprocess(**hydroprocess_kwargs)
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
# Report success

print(f'Hydronetwork(s) complete at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} (Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')



Egegik output location already exists at C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs\\TauDEM\Egegik
	----------
Egegik_layer
	----------
'Egegik'
	----------
Processing_Region = 'Egegik'
	----------
{} Processing Region already exists
	-----------
Begin Hydro Processing at
Pit Filled Elevation Grid already exists 
----------
Flow Direction Grid already exists
----------
D8 Contributing Area Grid Already Exists
----------
Streams Already Defined
----------
Stream Reach and Watershed already identified
----------
Watershed Grid already exported to shapefile
----------
Lakes output location already exists at C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\hydrography\\data\\Outputs\\TauDEM\Lakes
	----------
Lakes_layer
	----------
'Lakes'
	----------
Processing_Region = 'Lakes'
	----------
{} Processing Region already exists
	-----------
Begin Hydro Processing at
Pit Filled Elevation Grid already exists 
----------
Flow Direction Grid already exists
------

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ArcGISStabilityIndex.InitializeParameters.py, line 23)

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ParameterRegionTool.InitializeParameters.py, line 27)

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ArcGISStabilityIndex.InitializeParameters.py, line 23)

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ArcGISStabilityIndex.InitializeParameters.py, line 23)

...

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ParameterRegionTool.InitializeParameters.py, line 27)

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ParameterRegionTool.InitializeParameters.py, line 27)

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ParameterRegionTool.InitializeParameters.py, line 27)

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ParameterRegionTool.InitializeParameters.py, line 27)

TabError: inconsistent use of tabs and spaces in indentation (TauDEM Tools.tbx#ParameterRegionTool.InitializeParameters.py, line 27)